In [1]:
import open3d as o3d
import cl
import utils as ut
import numpy as np
from skimage.morphology import binary_dilation
import proc3d
import json
from PIL import Image
from utils import *
import glob
import os

In [2]:
class space_carving_2():
    def __init__(self, dataset_path):
        self.masks_files = sorted (glob.glob(os.path.join(dataset_path, 'masks', '*.png')) )#get all .png file names from folder path
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_mask(self,idx):                                                                                                                                         
        img = cv2.imread(self.masks_files[idx], cv2.IMREAD_GRAYSCALE)                                                                                                                                                                                                                                                                                                                                                                                     
        return img

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        im = self.load_mask(idx)
        self.space_carve(im, self.extrinsics[idx])
        
    def space_carve(self, mask, rt):
        #mask = im.copy() #get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

    
def create_pdict(name):
    pdict = {'name':name,'hlines':[],'cumulative_hlines':[],}
    return pdict

In [3]:
data_path = '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/'

In [24]:
plants = [create_pdict('000_2d'), create_pdict('001_2d'),
          create_pdict('003_2d'), create_pdict('006_2d'),
          create_pdict('009_2d'), create_pdict('124_2d'),
          create_pdict('195_2d')]

for plant in plants:
    sc = space_carving_2(os.path.join(data_path,plant['name']))
    #carve all rows cumulatively
    sc.reset()
    for i in range(720):      
        sc.carve(i)
        if (i+1)%180 == 0: #take cumulative chamfer distance from ground truth for each row
            ch_dist = sc.dist_to_gt()
            plant['cumulative_hlines'].append(ch_dist)
            #print(i,ch_dist)
    #first row of cumulative distance is the same as first row of independent distances
    plant['hlines'].append(plant['cumulative_hlines'][0])
    #carve rows independently
    for i in range(180,720,180):
        sc.reset()
        for j in range(180):
            sc.carve(i+j)
        ch_dist = sc.dist_to_gt()
        plant['hlines'].append(ch_dist)
        #print(i+j,ch_dist)

In [21]:
plants = json.load(open('cum_distances_plants.json'))

In [22]:
def test_carve(msc,imgs):
    msc.reset()
    for i in imgs:
        msc.carve(i)
        #print(np.unique(spc.sc.values()))
    cd = msc.dist_to_gt()
    return cd

def rnd_test(sc,n,set_size,sample_type='1d'):
    distances = []
    for i in range(n):
        if sample_type == '1d':
            images = np.random.randint(180, size=set_size)
        else:
            images = np.random.randint(720, size=set_size)
        distances.append( test_carve(sc,images) )
        print(f"{i}       ",end='\r')
    return np.mean(distances)

In [30]:
sc = space_carving_2(os.path.join(data_path,'003_2d'))

In [31]:
rnd_test(sc,100,20,'1d')

0.5011068110092415

In [24]:
for plant in plants.values():
    sc = space_carving_2(os.path.join(data_path,plant['name']))
    plant['rnd_1d'] = []
    plant['rnd_2d'] = []
    for i in range(10,151,10):
        print(plant['name'],i)
        plant['rnd_1d'].append(rnd_test(sc,100,i,'1d'))
        plant['rnd_2d'].append(rnd_test(sc,100,i,'2d'))

000_2d 10
000_2d 20
000_2d 30
000_2d 40
000_2d 50
000_2d 60
000_2d 70
000_2d 80
000_2d 90
000_2d 100
000_2d 110
000_2d 120
000_2d 130
000_2d 140
000_2d 150
001_2d 10
001_2d 20
001_2d 30
001_2d 40
001_2d 50
001_2d 60
001_2d 70
001_2d 80
001_2d 90
001_2d 100
001_2d 110
001_2d 120
001_2d 130
001_2d 140
001_2d 150
003_2d 10
003_2d 20
003_2d 30
003_2d 40
003_2d 50
003_2d 60
003_2d 70
003_2d 80
003_2d 90
003_2d 100
003_2d 110
003_2d 120
003_2d 130
003_2d 140
003_2d 150
006_2d 10
006_2d 20
006_2d 30
006_2d 40
006_2d 50
006_2d 60
006_2d 70
006_2d 80
006_2d 90
006_2d 100
006_2d 110
006_2d 120
006_2d 130
006_2d 140
006_2d 150
009_2d 10
009_2d 20
009_2d 30
009_2d 40
009_2d 50
009_2d 60
009_2d 70
009_2d 80
009_2d 90
009_2d 100
009_2d 110
009_2d 120
009_2d 130
009_2d 140
009_2d 150
124_2d 10
124_2d 20
124_2d 30
124_2d 40
124_2d 50
124_2d 60
124_2d 70
124_2d 80
124_2d 90
124_2d 100
124_2d 110
124_2d 120
124_2d 130
124_2d 140
124_2d 150
195_2d 10
195_2d 20
195_2d 30
195_2d 40
195_2d 50
195_2d 60
195_

In [25]:
with open('cum_distances_plants.json', 'w') as json_file:
            json.dump(plants, json_file)